##Practical Class Activity 1


Postgres Installation

In [ ]:
# Install postgresql server
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql
!sudo service postgresql start

# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"

# Setup a database with name `tfio_demo` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS enterprise;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE enterprise;'

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 11.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package cron.
(Reading database ... 144793 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128.1ubuntu1_amd64.deb ...
Unpacking cron (3.0pl1-128.1ubuntu1) ...
Selecting previously unselected package logrotate.
Preparing to unpack .../01-logrotate_3.11.0-0.1ubuntu1_amd64.deb ...
Unpacking logrotate (3.11.0-0.1ubuntu1) ...
Selecting previously unselected package netbase.
Preparing to unpack .../02-netbase_5.4_all.deb ...
Unpacking netbase (5.4) ...
Preparing to unpack .../03-libpq

In [ ]:
%env TFIO_DEMO_DATABASE_NAME=enterprise
%env TFIO_DEMO_DATABASE_HOST=localhost
%env TFIO_DEMO_DATABASE_PORT=5432
%env TFIO_DEMO_DATABASE_USER=postgres
%env TFIO_DEMO_DATABASE_PASS=postgres

env: TFIO_DEMO_DATABASE_NAME=enterprise
env: TFIO_DEMO_DATABASE_HOST=localhost
env: TFIO_DEMO_DATABASE_PORT=5432
env: TFIO_DEMO_DATABASE_USER=postgres
env: TFIO_DEMO_DATABASE_PASS=postgres


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!PGPASSWORD=$TFIO_DEMO_DATABASE_PASS psql -q -h $TFIO_DEMO_DATABASE_HOST -p $TFIO_DEMO_DATABASE_PORT -U $TFIO_DEMO_DATABASE_USER -d $TFIO_DEMO_DATABASE_NAME -f '/content/northwind.sql'
!psql postgres < '/content/drive/Shareddrives/modulo4/Tarefa 1: Aula Prática 1 (SQL)/northwind.sql'

In [ ]:
# set connection
%load_ext sql
%config SqlMagic.feedback=False
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


'Connected: @postgres'

In [ ]:
#1. Top 10 most expensive products
df = %sql SELECT distinct products.product_name as ten_most_expensive_products,\
            products.unit_price from products \
            ORDER BY products.unit_price desc \
            limit 10
df

 * postgresql+psycopg2://@/postgres


,ten_most_expensive_products,unit_price
0,Côte de Blaye,263.50
1,Thüringer Rostbratwurst,123.79
2,Mishi Kobe Niku,97.00
3,Sir Rodney's Marmalade,81.00
4,Carnarvon Tigers,62.50
5,Raclette Courdavault,55.00
6,Manjimup Dried Apples,53.00
7,Tarte au sucre,49.30
8,Ipoh Coffee,46.00
9,Rössle Sauerkraut,45.60


In [ ]:
#2. Quarterly order
df = %sql Select TO_CHAR(order_date, 'YYYY') as ano, \
  CASE WHEN TO_CHAR(order_date, 'MM') <= '03' THEN '1-trimestre' \
  WHEN TO_CHAR(order_date, 'MM') > '03' and TO_CHAR(order_date, 'MM') <= '06' THEN '2-trimestre' \
  WHEN TO_CHAR(order_date, 'MM') > '06' and TO_CHAR(order_date, 'MM') <= '09' THEN '3-trimestre' \
  ELSE '4-trimestre' END trimestre,\
  Round(sum(unit_price * quantity - (unit_price * quantity * discount))) from order_details \
  join orders on order_details.order_id = orders.order_id \
  group by ano, trimestre \
  order by ano, trimestre

df

 * postgresql+psycopg2://@/postgres


,ano,trimestre,round
0,1996,3-trimestre,79729.0
1,1996,4-trimestre,128355.0
2,1997,1-trimestre,138289.0
3,1997,2-trimestre,143177.0
4,1997,3-trimestre,153938.0
5,1997,4-trimestre,181681.0
6,1998,1-trimestre,298492.0
7,1998,2-trimestre,142132.0


In [ ]:
#3. Extended Order Details
df = %sql select order_details.order_id, order_details.product_id, products.product_name, \
            order_details.unit_price, order_details.quantity, order_details.discount \
            from products inner join order_details on products.product_id = order_details.product_id \
            order by order_details.order_id
df

 * postgresql+psycopg2://@/postgres


,order_id,product_id,product_name,unit_price,quantity,discount
0,10248,11,Queso Cabrales,14.00,12,0.00
1,10248,42,Singaporean Hokkien Fried Mee,9.80,10,0.00
2,10248,72,Mozzarella di Giovanni,34.80,5,0.00
3,10249,14,Tofu,18.60,9,0.00
4,10249,51,Manjimup Dried Apples,42.40,40,0.00
...,...,...,...,...,...,...
2150,11077,64,Wimmers gute Semmelknödel,33.25,2,0.03
2151,11077,66,Louisiana Hot Spiced Okra,17.00,1,0.00
2152,11077,73,Röd Kaviar,15.00,2,0.01
2153,11077,75,Rhönbräu Klosterbier,7.75,4,0.00


In [ ]:
#4. Products above average value
df = %sql select product_name, unit_price \
        from products where unit_price > (select AVG(unit_price) from products) \
        order by unit_price desc;
df

 * postgresql+psycopg2://@/postgres


,product_name,unit_price
0,Côte de Blaye,263.50
1,Thüringer Rostbratwurst,123.79
2,Mishi Kobe Niku,97.00
3,Sir Rodney's Marmalade,81.00
4,Carnarvon Tigers,62.50
5,Raclette Courdavault,55.00
6,Manjimup Dried Apples,53.00
7,Tarte au sucre,49.30
8,Ipoh Coffee,46.00
9,Rössle Sauerkraut,45.60


In [ ]:
#5. Order subtotal
df = %sql select order_id, \
            sum(unit_price * quantity * discount) as sub_total_with_discount, \
            sum(unit_price * quantity) as sub_total, \
            sum(unit_price * quantity - (unit_price * quantity * discount)) as total_with_discount \
            from order_details \
            group by order_id;
df

 * postgresql+psycopg2://@/postgres


,order_id,sub_total_with_discount,sub_total,total_with_discount
0,11038,18.400000,750.999998,732.599998
1,10782,0.000000,12.500000,12.500000
2,10725,0.000000,287.799995,287.799995
3,10423,0.000000,1020.000000,1020.000000
4,10518,0.000000,4150.050007,4150.050007
...,...,...,...,...
825,10707,63.000003,1704.000000,1640.999997
826,10826,0.000000,730.000000,730.000000
827,10371,18.240000,91.199999,72.959999
828,10575,0.000000,2147.399986,2147.399986


In [ ]:
#6. Sales per year
df = %sql Select TO_CHAR(order_date, 'YYYY') as year, \
            sum(unit_price * quantity - (unit_price * quantity * discount)) from order_details \
            join orders on order_details.order_id = orders.order_id \
            group by year
df

 * postgresql+psycopg2://@/postgres


,year,sum
0,1997,617085.202393
1,1998,440623.865278
2,1996,208083.970983


In [ ]:
df.to_csv('consulta6.csv', sep = '|')

In [ ]:
#7. Products sold in 1996
df = %sql Select orders_by_product.product_id, products.product_name, (price * quantity) as product_total_value from products \
        join (\
          Select order_details.product_id, sum(order_details.unit_price) as price,\
           sum(order_details.quantity) as quantity \
           from order_details \
           join orders on order_details.order_id = orders.order_id \
           where TO_CHAR(orders.order_date, 'YYYY') = '1996' \
           group by order_details.product_id) as orders_by_product \
        on products.product_id = orders_by_product.product_id
df

 * postgresql+psycopg2://@/postgres


,product_id,product_name,product_total_value
0,1,Chai,10800.000191
1,2,Chang,27481.597931
2,3,Aniseed Syrup,240.000000
3,4,Chef Anton's Cajun Seasoning,9416.000000
4,5,Chef Anton's Gumbo Mix,8772.000000
...,...,...,...
69,73,Röd Kaviar,300.000000
70,74,Longlife Tofu,5640.000000
71,75,Rhönbräu Klosterbier,5208.000183
72,76,Lakkalikööri,14716.800446


In [ ]:
df.to_csv('consulta7.csv', sep = '|')

In [ ]:
#8.Sales by Category
df = %sql select distinct products.category_id, categories.category_name,\
 sum(order_details.unit_price * order_details.quantity - (order_details.unit_price * order_details.quantity * order_details.discount)) as total \
 from order_details inner join \
 products on products.product_id = order_details.product_id\
 inner join categories on products.category_id = categories.category_id \
 group by products.category_id, categories.category_name \
 order by products.category_id

df

 * postgresql+psycopg2://@/postgres


,category_id,category_name,total
0,1,Beverages,267868.179786
1,2,Condiments,106047.084610
2,3,Confections,167357.225473
3,4,Dairy Products,234507.284531
4,5,Grains/Cereals,95744.587355
5,6,Meat/Poultry,163022.360269
6,7,Produce,99984.580074
7,8,Seafood,131261.736556


In [ ]:
df.to_csv('consulta8.csv', sep = '|')